In [25]:
import pandas as pd
import numpy as np
import os

In [26]:
df = pd.read_csv('../Desktop/data_2024_2015.csv')
weather = pd.read_csv('../Desktop/weather_data.csv')
df = df.sort_values(by=['setmana_epidemiologica', 'any'])
df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
df['data'] = df['data'].dt.date
weather['DATA_LECTURA'] = pd.to_datetime(weather['DATA_LECTURA'], format='%d/%m/%Y %I:%M:%S %p')
weather['DATA_LECTURA'] = weather['DATA_LECTURA'].dt.date
weather
# 32 = temperature
# 33 = humidity

,ID,CODI_VARIABLE,DATA_LECTURA,VALOR_LECTURA
0,CW322710211630,32,2021-10-27,14.5
1,DL320702221000,32,2022-02-07,15.0
2,DL330702221000,33,2022-02-07,52.0
3,UW322505221200,32,2022-05-25,19.1
4,UW332505221200,33,2022-05-25,66.0
...,...,...,...,...
65732693,XG321503211030,32,2021-03-15,15.0
65732694,XG331503211000,33,2021-03-15,44.0
65732695,XG331503211030,33,2021-03-15,39.0
65732696,WU320407211030,32,2021-07-04,25.8


In [27]:
# separate the weather by temperature and humidity
temperature = weather[weather['CODI_VARIABLE'] == 32]
temperature = temperature.drop(columns=['CODI_VARIABLE', 'ID'])

humidity = weather[weather['CODI_VARIABLE'] == 33]
humidity = humidity.drop(columns=['CODI_VARIABLE', 'ID'])
print(humidity['DATA_LECTURA'].duplicated().sum())
temperature_daily = temperature.groupby('DATA_LECTURA').mean().reset_index()
humidity_daily = humidity.groupby('DATA_LECTURA').mean().reset_index()
humidity_daily

32863047


,DATA_LECTURA,VALOR_LECTURA
0,2015-01-01,63.728571
1,2015-01-02,57.353690
2,2015-01-03,58.559524
3,2015-01-04,65.704405
4,2015-01-05,73.862381
...,...,...
3768,2025-04-29,73.754863
3769,2025-04-30,70.829397
3770,2025-05-01,60.591195
3771,2025-05-02,75.486881


In [28]:
# group by 
df = df.groupby(['data', 'setmana_epidemiologica', 'any', 'nom_regio', 'diagnostic', 'grup_edat'], as_index=False).agg({'casos': 'sum', 'poblacio': 'sum'})
df

,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos,poblacio
0,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,0,7,221
1,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,1 i 2,12,649
2,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,10 a 14,3,238
3,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,15 a 19,2,552
4,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,20 a 24,6,918
...,...,...,...,...,...,...,...,...
2469525,2025-04-27,17,2025,Terres de l'Ebre,Grip,35 a 39,1,465
2469526,2025-04-27,17,2025,Terres de l'Ebre,Grip,5 a 9,1,325
2469527,2025-04-27,17,2025,Terres de l'Ebre,Grip,55 a 59,1,563
2469528,2025-04-27,17,2025,Terres de l'Ebre,Grip,65 a 69,1,389


In [29]:
# merge temperature, humidity and df using 'data' and 'DATA_LECTURA' as keys
# merge temperature, humidity and df using 'data' and 'DATA_LECTURA' as keys
df = df.merge(temperature_daily, left_on='data', right_on='DATA_LECTURA', how='left')
df = df.merge(humidity_daily, left_on='data', right_on='DATA_LECTURA', how='left')
df = df.drop(columns=['DATA_LECTURA_x', 'DATA_LECTURA_y'])
df = df.rename(columns={'VALOR_LECTURA_x': 'temperature', 'VALOR_LECTURA_y': 'humidity'})
df

,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos,poblacio,temperature,humidity
0,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,0,7,221,7.770576,89.042837
1,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,1 i 2,12,649,7.770576,89.042837
2,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,10 a 14,3,238,7.770576,89.042837
3,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,15 a 19,2,552,7.770576,89.042837
4,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,20 a 24,6,918,7.770576,89.042837
...,...,...,...,...,...,...,...,...,...,...
2469525,2025-04-27,17,2025,Terres de l'Ebre,Grip,35 a 39,1,465,12.629131,74.424007
2469526,2025-04-27,17,2025,Terres de l'Ebre,Grip,5 a 9,1,325,12.629131,74.424007
2469527,2025-04-27,17,2025,Terres de l'Ebre,Grip,55 a 59,1,563,12.629131,74.424007
2469528,2025-04-27,17,2025,Terres de l'Ebre,Grip,65 a 69,1,389,12.629131,74.424007


In [30]:
df['semana_sin'] = np.sin(2 * np.pi * df['setmana_epidemiologica'] / 53)
df['semana_cos'] = np.cos(2 * np.pi * df['setmana_epidemiologica'] / 53)

In [31]:
for lag in range(1, 4):  # Crear lags de 1 a 4 semanas
    df[f'casos_t{lag}'] = df.groupby(['nom_regio', 'diagnostic', 'grup_edat'])['casos'].shift(lag)
df[['casos_t1', 'casos_t2', 'casos_t3']] = df[['casos_t1', 'casos_t2', 'casos_t3']].fillna(0)
df

,data,setmana_epidemiologica,any,nom_regio,diagnostic,grup_edat,casos,poblacio,temperature,humidity,semana_sin,semana_cos,casos_t1,casos_t2,casos_t3
0,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,0,7,221,7.770576,89.042837,0.118273,0.992981,0.0,0.0,0.0
1,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,1 i 2,12,649,7.770576,89.042837,0.118273,0.992981,0.0,0.0,0.0
2,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,10 a 14,3,238,7.770576,89.042837,0.118273,0.992981,0.0,0.0,0.0
3,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,15 a 19,2,552,7.770576,89.042837,0.118273,0.992981,0.0,0.0,0.0
4,2016-01-04,1,2016,Alt Pirineu i Aran,Altres IRA,20 a 24,6,918,7.770576,89.042837,0.118273,0.992981,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469525,2025-04-27,17,2025,Terres de l'Ebre,Grip,35 a 39,1,465,12.629131,74.424007,0.902798,-0.430065,1.0,1.0,2.0
2469526,2025-04-27,17,2025,Terres de l'Ebre,Grip,5 a 9,1,325,12.629131,74.424007,0.902798,-0.430065,1.0,1.0,2.0
2469527,2025-04-27,17,2025,Terres de l'Ebre,Grip,55 a 59,1,563,12.629131,74.424007,0.902798,-0.430065,1.0,1.0,1.0
2469528,2025-04-27,17,2025,Terres de l'Ebre,Grip,65 a 69,1,389,12.629131,74.424007,0.902798,-0.430065,1.0,1.0,1.0


In [32]:
# Eliminar columnas no necesarias
df = df.drop(columns=['setmana_epidemiologica'])

# Reordenar columnas: poner semana_sin y semana_cos al inicio
cols = ['semana_sin', 'semana_cos'] + [col for col in df.columns if col not in ['semana_sin', 'semana_cos']]
df = df[cols]
df

,semana_sin,semana_cos,data,any,nom_regio,diagnostic,grup_edat,casos,poblacio,temperature,humidity,casos_t1,casos_t2,casos_t3
0,0.118273,0.992981,2016-01-04,2016,Alt Pirineu i Aran,Altres IRA,0,7,221,7.770576,89.042837,0.0,0.0,0.0
1,0.118273,0.992981,2016-01-04,2016,Alt Pirineu i Aran,Altres IRA,1 i 2,12,649,7.770576,89.042837,0.0,0.0,0.0
2,0.118273,0.992981,2016-01-04,2016,Alt Pirineu i Aran,Altres IRA,10 a 14,3,238,7.770576,89.042837,0.0,0.0,0.0
3,0.118273,0.992981,2016-01-04,2016,Alt Pirineu i Aran,Altres IRA,15 a 19,2,552,7.770576,89.042837,0.0,0.0,0.0
4,0.118273,0.992981,2016-01-04,2016,Alt Pirineu i Aran,Altres IRA,20 a 24,6,918,7.770576,89.042837,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469525,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,35 a 39,1,465,12.629131,74.424007,1.0,1.0,2.0
2469526,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,5 a 9,1,325,12.629131,74.424007,1.0,1.0,2.0
2469527,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,55 a 59,1,563,12.629131,74.424007,1.0,1.0,1.0
2469528,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,65 a 69,1,389,12.629131,74.424007,1.0,1.0,1.0


In [33]:
# remove any rows with 'No disponible' in the 'nom_regio' column
df = df[df['nom_regio'] != 'No disponible']

# move the casos column to the end
casos = df.pop('casos')
df['casos'] = casos
df
# remove poblacio column
df = df.drop(columns=['poblacio'])

# remove all the rows with 'any' below 2020
df = df[df['any'] >= 2020]
df

/var/folders/td/gn83nbyd2fvc_t_wg8k3c08w0000gn/T/ipykernel_96472/2429862665.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['casos'] = casos


,semana_sin,semana_cos,data,any,nom_regio,diagnostic,grup_edat,temperature,humidity,casos_t1,casos_t2,casos_t3,casos
924716,0.118273,0.992981,2019-12-30,2020,Alt Pirineu i Aran,Altres IRA,0,5.622894,82.393229,2.0,1.0,5.0,1
924717,0.118273,0.992981,2019-12-30,2020,Alt Pirineu i Aran,Altres IRA,1 i 2,5.622894,82.393229,2.0,9.0,5.0,7
924718,0.118273,0.992981,2019-12-30,2020,Alt Pirineu i Aran,Altres IRA,10 a 14,5.622894,82.393229,3.0,1.0,1.0,3
924719,0.118273,0.992981,2019-12-30,2020,Alt Pirineu i Aran,Altres IRA,15 a 19,5.622894,82.393229,1.0,1.0,7.0,3
924720,0.118273,0.992981,2019-12-30,2020,Alt Pirineu i Aran,Altres IRA,20 a 24,5.622894,82.393229,2.0,2.0,2.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2469525,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,35 a 39,12.629131,74.424007,1.0,1.0,2.0,1
2469526,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,5 a 9,12.629131,74.424007,1.0,1.0,2.0,1
2469527,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,55 a 59,12.629131,74.424007,1.0,1.0,1.0,1
2469528,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,Grip,65 a 69,12.629131,74.424007,1.0,1.0,1.0,1


In [34]:
print(df['diagnostic'].unique())

['Altres IRA' 'Bronquiolitis' 'Faringoamigdalitis'
 'Faringoamigdalitis estreptocòccica' 'Grip' 'Impetigen' 'Pneumònia'
 'COVID-19' 'Escarlatina']


In [35]:
def process_diagnostic(df, diagnostic, new_column_name):
    df_filtered = df[df['diagnostic'] == diagnostic]
    df_filtered = df_filtered.drop(columns=['diagnostic'])
    df_filtered = df_filtered.rename(columns={'casos': new_column_name})
    return df_filtered

diagnostics = {
    'COVID-19': 'casos_covid',
    'Bronquiolitis': 'casos_bronquio',
    'Faringoamigdalitis': 'casos_fa',
    'Faringoamigdalitis estreptococica': 'casos_fa_es',
    'Grip': 'casos_grip',
    'Impetigen': 'casos_imp',
    'Escarlatina': 'casos_escarlatina',
    'Pneumònia': 'casos_pneumonia',
    'Altres IRA': 'casos_ira'
}

processed_dfs = {}
for diagnostic, new_column_name in diagnostics.items():
    processed_dfs[diagnostic] = process_diagnostic(df, diagnostic, new_column_name)

# Access individual dataframes like this:
df_covid = processed_dfs['COVID-19']
df_bronquio = processed_dfs['Bronquiolitis']
df_fa = processed_dfs['Faringoamigdalitis']
df_fa_es = processed_dfs['Faringoamigdalitis estreptococica']
df_grip = processed_dfs['Grip']
df_imp = processed_dfs['Impetigen']
df_escarlatina = processed_dfs['Escarlatina']
df_pneumonia = processed_dfs['Pneumònia']
df_ira = processed_dfs['Altres IRA']

# save the dataframes to csv files
os.makedirs('datasets_IRA', exist_ok=True)
df_covid.to_csv('datasets_IRA/covid.csv', index=False)
df_bronquio.to_csv('datasets_IRA/bronquilitis.csv', index=False)
df_fa.to_csv('datasets_IRA/faringoamig.csv', index=False)
df_fa_es.to_csv('datasets_IRA/faringoamig_estrepto.csv', index=False)
df_grip.to_csv('datasets_IRA/grip.csv', index=False)
df_imp.to_csv('datasets_IRA/impetigen.csv', index=False)
df_escarlatina.to_csv('datasets_IRA/escarlatina.csv', index=False)
df_pneumonia.to_csv('datasets_IRA/pneumonia.csv', index=False)
df_ira.to_csv('datasets_IRA/altres_ira.csv', index=False)



In [36]:
df_covid

,semana_sin,semana_cos,data,any,nom_regio,grup_edat,temperature,humidity,casos_t1,casos_t2,casos_t3,casos_covid
924788,0.118273,0.992981,2019-12-30,2020,Barcelona Ciutat,30 a 34,5.622894,82.393229,0.0,0.0,0.0,1
924891,0.118273,0.992981,2019-12-30,2020,Barcelona Metropolitana Nord,30 a 34,5.622894,82.393229,0.0,0.0,0.0,1
925098,0.118273,0.992981,2019-12-30,2020,Camp de Tarragona,60 a 64,5.622894,82.393229,0.0,0.0,0.0,1
925265,0.118273,0.992981,2019-12-30,2020,Girona,30 a 34,5.622894,82.393229,1.0,0.0,0.0,1
925477,0.118273,0.992981,2019-12-30,2020,Penedès,45 a 49,5.622894,82.393229,0.0,0.0,0.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
2469397,0.902798,-0.430065,2025-04-27,2025,Lleida,30 a 34,12.629131,74.424007,1.0,1.0,1.0,1
2469398,0.902798,-0.430065,2025-04-27,2025,Lleida,80 o més,12.629131,74.424007,1.0,1.0,1.0,1
2469510,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,10 a 14,12.629131,74.424007,1.0,1.0,1.0,1
2469511,0.902798,-0.430065,2025-04-27,2025,Terres de l'Ebre,35 a 39,12.629131,74.424007,1.0,1.0,1.0,1
